In [1]:
import re
import numpy as np
import pandas as pd
import PyPDF2
from io import BytesIO
from selenium import webdriver
from urllib.request import urlopen

# Functions

### Read PDF Content from URL

In [2]:
def read_data(url):
    pdf_content = urlopen(link)
    pdfFileObj = BytesIO(pdf_content.read())
    pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
    pageObj = pdfReader.getPage(2)
    
    return pageObj.extractText()

### Parse the PDF Content

In [3]:
def parse_data(data):
    parsed = data
    parsed = parsed[parsed.index("Istanbul"):]
    parsed = parsed[:parsed.index("Turkey")]
    return re.findall(r"([A-Z][A-Za-z\s]*)\n \n(\d*)\n \n", parsed)

### Create a DataFrame from parsed content

In [4]:
def get_row(parsed, dt):
    df_data = {"Date": pd.to_datetime(dt, format='%d%m%Y', errors='ignore')}
    for pair in parsed:
        df_data[pair[0].replace('\n', '')] = [pair[1]]
    return pd.DataFrame(df_data)

# Program

### Get PDF links from the page

In [5]:
driver = webdriver.Firefox()
driver.get("https://sbsgm.saglik.gov.tr/TR,66559/gunluk-rapor--daily-report.html")
table = driver.find_element_by_class_name("table")
elems = table.find_elements_by_tag_name("a")
links = [elem.get_attribute('href') for elem in elems]
del links[1::2] # Get only half of the links (links of the english version)

### Create a DataFrame from parsed content

In [6]:
df = pd.DataFrame()
for link in links:
    data = read_data(link)
    date = re.search("-(\d*)-eng.html", link).group(1)
    parsed = parse_data(data)
    df = pd.concat([df, get_row(parsed, date)])